<img src="assets/logo.jpg" alt="HPC Builder Logo" width="64"/>

# HPC Container Builder

ARC - Aschaffenburger Research Cluster<br>
Version 2024.5.1<br>
_by Kat 2024_

#### Setup docker build environment

First, you'll need to setup some mandatory information before start using the HPC Builder. Create a file **__hpc_setup.yaml** in your workspace root folder and fill in all information:

**Also:** Just put your yaml file in gitignore to avoid pushing that info to the repository.

In [1]:
from io import StringIO
import pandas as pd
import subprocess
import yaml
import hpctk

# load user HPC settings yaml
with open('../__hpc_creds.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Access the variables
lab = config["lab"]
user = config["user"]
workspace_dir = config["workspace_dir"]
container_name = config["container_name"]
container_tag = config["container_tag"]
docker_file = config["docker_file"]
registry_host = config["registry_host"]
hpc_mn = config["hpc_mn"]
hpc_mn_project_base = config["hpc_mn_project_base"]
ssh_creds = config["ssh_creds"]

hpctk.print_hpc_setup_yaml(lab, user, container_name, container_tag, docker_file, registry_host, hpc_mn, ssh_creds, workspace_dir, hpc_mn_project_base)

Fact,Value
lab,aghm
user,u230292
container_name,datagen
container_tag,latest
docker_file,Dockerfile
registry_host,DT-IW-20-028:5000
hpc_mn,10.32.118.9
ssh_creds,"{'port': '44100', 'ssh_key_name': 'hpc_u230292', 'ssh_path': '~/.ssh', 'user': 'u230292'}"
workspace_dir,data_gen
hpc_mn_project_base,/home_cu/u230292/Projects


#### Build and tag docker image for HPC

You'll need to build your docker image first. A build can take up to **10 minutes** or more so just be patient. A build might error out. **Read the error messages and don't ignore any warnings.** If your docker image was built fine, a tag for a specific docker registry is mandatory.

In [21]:
try:
    print(f"{hpctk.STATE_WORK} Build docker image local/{lab}_{user}_{container_name}:{container_tag}...")
    hpctk.run(["docker", "build", "-f", f"{docker_file}", "-t", f"local/{lab}_{user}_{container_name}:{container_tag}", "."])
    print(f"{hpctk.STATE_WORK} Tag docker image {registry_host}/{lab}_{user}_{container_name}:{container_tag}...")
    hpctk.run(["docker", "image", "tag", f"local/{lab}_{user}_{container_name}:{container_tag}", f"{registry_host}/{lab}_{user}_{container_name}:{container_tag}"])
    print(f"{hpctk.STATE_DONE} DONE!")

    result = hpctk.run(["docker", "image", "ls", "--format", "REPOSITORY\tTAG\tIMAGE_ID\tSIZE\n{{.Repository}}\t{{.Tag}}\t{{.ID}}\t{{.Size}}", f"{registry_host}/{lab}_{user}_{container_name}:{container_tag}"], stdout=False)
    hpctk.show_tab_styled(result.stdout, 0, hpctk.highlight_tags, columns=['TAG'])

except subprocess.CalledProcessError as e: 
    print(f"\n{hpctk.STATE_FAIL} Failed: {e.returncode}\n")
    print(f":: Error message: {e.stderr}")

🚧 Build docker image local/aghm_u230292_datagen:latest...

🚧 Tag docker image 10.32.118.10:5000/aghm_u230292_datagen:latest...

✅ DONE!


,REPOSITORY,TAG,IMAGE_ID,SIZE
0,10.32.118.10:5000/aghm_u230292_datagen,latest,13b0e07814fe,9.45GB


#### Build and tag docker image for development

Run this to create a local container image for development.

In [22]:
try:
    print(f"{hpctk.STATE_WORK} Build docker image local/{container_name}:{container_tag}...")
    hpctk.run(["docker", "build", "-f", f"{docker_file}", "-t", f"local/{container_name}:{container_tag}", "."])
    print(f"{hpctk.STATE_DONE} DONE!")

    result = hpctk.run(["docker", "image", "ls", "--format", "REPOSITORY\tTAG\tIMAGE_ID\tSIZE\n{{.Repository}}\t{{.Tag}}\t{{.ID}}\t{{.Size}}", f"{registry_host}/{lab}_{user}_{container_name}:{container_tag}"], stdout=False)
    hpctk.show_tab_styled(result.stdout, 0, hpctk.highlight_tags, columns=['TAG'])

except subprocess.CalledProcessError as e:
    print(f"\n{hpctk.STATE_FAIL} Failed: {e.returncode}\n")
    print(f":: Error message: {e.stderr}")

🚧 Build docker image local/datagen:latest...



✅ DONE!


,REPOSITORY,TAG,IMAGE_ID,SIZE
0,10.32.118.10:5000/aghm_u230292_datagen,latest,13b0e07814fe,9.45GB


#### Push docker image to HPC

In [25]:
try:
    print(f"{hpctk.STATE_WORK} Push docker image {registry_host}/{lab}_{user}_{container_name}:{container_tag}...")
    hpctk.run(["docker", "push", f"{registry_host}/{lab}_{user}_{container_name}:{container_tag}"], stdout=False)
    print(f"\n{hpctk.STATE_DONE} DONE!\n")
    
    hpctk.run(["curl", "-X", "GET", f"https://{registry_host}/v2/{lab}_{user}_{container_name}/tags/list"])

except subprocess.CalledProcessError as e:
    print(f"\n{hpctk.STATE_FAIL} Failed: {e.returncode}\n")
    print(f":: Error message: {e.stderr}")

🚧 Push docker image 10.32.118.10:5000/aghm_u230292_datagen:latest...

🤢 Failed: 1

:: Error message: Get "https://10.32.118.10:5000/v2/": net/http: request canceled while waiting for connection (Client.Timeout exceeded while awaiting headers)



#### Push code to HPC

In [6]:
try:
    ssh_port = ssh_creds["port"]
    ssh_key_file = f"{ssh_creds['ssh_path']}/{ssh_creds['ssh_key_name']}"
    ssh_user = ssh_creds["user"]

    print(f"{hpctk.STATE_WORK} Push code to HPC {ssh_user}@{hpc_mn}:{hpc_mn_project_base}...")
    hpctk.run(["rsync", "-av", f"--rsh=ssh -p {ssh_port} -i {ssh_key_file}", f"../{workspace_dir}", f"{ssh_user}@{hpc_mn}:{hpc_mn_project_base}/"])
    print(f"\n{hpctk.STATE_DONE} DONE!\n")

except subprocess.CalledProcessError as e:
    print(f"\n{hpctk.STATE_FAIL} Failed: {e.returncode}\n")
    print(f"Error message: {e.stderr}")

🚧 Push code to HPC u230292@10.32.118.9:/home_cu/u230292/Projects...
sending incremental file list
data_gen/hpc/run_LARs.bash

sent 1,494 bytes  received 75 bytes  1,046.00 bytes/sec
total size is 2,111,598  speedup is 1,345.82


✅ DONE!

